In [6]:
import requests
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from logging import getLogger, basicConfig, INFO, WARN
from bs4 import BeautifulSoup
import os
import re
from urllib.parse import urljoin
LOGGER = getLogger(__name__)
basicConfig(level=WARN)
DATE_FORMAT = '%Y%m%d'

SEARCH_URL = 'https://www.mhlw.go.jp/stf/houdou/houdou_list_{}.html'
DOMAIN = 'https://www.mhlw.go.jp'

In [52]:
start = datetime.strptime('20200201', DATE_FORMAT)
end = datetime.strptime("20200228", DATE_FORMAT)
date_range = [end - timedelta(days=x) for x in range((end-start).days+1)]

def get_page_list(yyyymm):
    url = SEARCH_URL.format(yyyymm)
    r = requests.get(url)
    r.raise_for_status()

    soup = BeautifulSoup(r.content)
    targets = soup.find_all(text=re.compile("新型コロナウイルス感染症の現在の状況"))
    link_list = [elem.parent.parent.parent for elem in targets]
    
    return [link.get('href') for link in link_list]

def convert_reiwa(str_):
    cleaned_str = (str_
                   .replace('令和２年', '2020/')
                   .replace('月', '/')
                   .replace('日版', '')                   
                   .replace('（', '')                   
                   .replace('）', '')                   
                   .translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
                  )
    return datetime.strptime(cleaned_str, '%Y/%m/%d')

In [56]:
output = get_page_list('202004')

In [58]:
strs = "（令和２年5月6日版）", "（令和２年５月１日版）"
[convert_reiwa(str_) for str_ in strs]

[datetime.datetime(2020, 5, 6, 0, 0), datetime.datetime(2020, 5, 1, 0, 0)]